In [5]:
import os
import json
import requests
from openai import OpenAI
import time
from dotenv import load_dotenv, find_dotenv

# API keys are stored in Google Colab's Secret Manager
load_dotenv(find_dotenv())
FMP_API_KEY: str = os.environ.get("FMP_API_KEY")
OPENAI_API_KEY: str = os.environ.get("OPENAI_API_KEY")
# print("OpenAI API Key ===>",OPENAI_API_KEY)
# print("FMP API Key ===>",FMP_API_KEY)



In [6]:
from openai import OpenAI
client : OpenAI = OpenAI()

# Step 1: Defining Financial Functions



In [19]:
# Define financial statement functions
def get_income_statement(ticker : str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/income-statement/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())

def get_balance_sheet(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


def get_cash_flow_statement(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


def get_key_metrics(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


def get_financial_ratios(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/ratios-ttm/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


def get_financial_growth(ticker: str) -> str:
    url = f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?period=annual&apikey={FMP_API_KEY}"
    response = requests.get(url)
    return json.dumps(response.json())


# Step 2: Map available functions

In [20]:
# Map available functions
available_functions = {
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_cash_flow_statement": get_cash_flow_statement,
    "get_key_metrics": get_key_metrics,
    "get_financial_ratios": get_cash_flow_statement,
    "get_financial_growth": get_financial_ratios
}
display(dict(available_functions))

{'get_income_statement': <function __main__.get_income_statement(ticker: str) -> str>,
 'get_balance_sheet': <function __main__.get_balance_sheet(ticker: str) -> str>,
 'get_cash_flow_statement': <function __main__.get_cash_flow_statement(ticker: str) -> str>,
 'get_key_metrics': <function __main__.get_key_metrics(ticker: str) -> str>,
 'get_financial_ratios': <function __main__.get_cash_flow_statement(ticker: str) -> str>,
 'get_financial_growth': <function __main__.get_financial_ratios(ticker: str) -> str>}

# Step 3: Creating the Assistant

In [21]:
function_schema = [
    {"type": "code_interpreter"},
    {
        {
            "type": "function",
            "function": {
                "name": "get_income_statement",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_balance_sheet",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                    },
                    "required": ["ticker"],
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_cash_flow_statement",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                    },
                    "required": ["ticker"],
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_key_metrics",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_financial_ratios",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                    },
                    "required": ["ticker"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "get_financial_growth",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string"},
                    },
                    "required": ["ticker"],
                },
            },
        },
    },
]

In [22]:
from openai.types.beta import Assistant

assistant: Assistant = client.beta.assistants.create(
    name="Financial Advisor",
    instructions="Act as a financial advisor by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include providing an investement advise by analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends.",
    model="gpt-3.5-turbo-1106",
    tools=function_schema
)

display(dict(assistant))

{'id': 'asst_Bu0sKf4zsGiNBS4wscEMbdct',
 'created_at': 1703589432,
 'description': None,
 'file_ids': [],
 'instructions': 'Act as a financial advisor by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include providing an investement advise by analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Financial Advisor',
 'object': 'assistant',
 'tools': [ToolFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}}, 'required': ['ticker']}, description=None), type='function'),
  ToolFunction(function=FunctionDefinition(name='get_balance_sheet', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}}, 'required': ['ticker']}, description=None), type='function'),
  ToolFunction(function=FunctionDefinition(name='get_cash_

# Step 4: Initiating a Thread

In [23]:

from openai.types.beta.thread import Thread

thread: Thread = client.beta.threads.create()

display(dict(thread))

{'id': 'thread_H7ikU3QleXqjpffxnj0zpS2Y',
 'created_at': 1703589434,
 'metadata': {},
 'object': 'thread'}

# Step 5: Adding Messages to the Thread

In [24]:
from openai.types.beta.threads.thread_message import ThreadMessage

message : ThreadMessage = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I have a lot of money to invest. Can you suggest if it is better to invest in Apple or Microsoft?",
)

display(dict(message))

{'id': 'msg_8ShOBBwEqjAsZuAvVWn7dubm',
 'assistant_id': None,
 'content': [MessageContentText(text=Text(annotations=[], value='I have a lot of money to invest. Can you suggest if it is better to invest in Apple or Microsoft?'), type='text')],
 'created_at': 1703589437,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_H7ikU3QleXqjpffxnj0zpS2Y'}

# Step 6: Running and Monitoring the Assistant

## 6.1 Start a Run

In [33]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
    thread_id=thread.id, assistant_id=assistant.id
)

display(dict(run))

{'id': 'run_jHxHhqHIfnx6vnZd4ApECelB',
 'assistant_id': 'asst_Bu0sKf4zsGiNBS4wscEMbdct',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1703591289,
 'expires_at': 1703591889,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'Act as a financial advisor by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include providing an investement advise by analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_H7ikU3QleXqjpffxnj0zpS2Y',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}}, 'required': ['ticker']}, description=None), type='function'),
  ToolAssistantToolsFu

## 6.2 Monitor and Manage the Run:

In [34]:
import time

while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            if function_name in available_functions:
                print("Calling ", function_name)
                function_to_call = available_functions[function_name]
                output = function_to_call(**function_args)
                tool_outputs.append(
                    {
                        "tool_call_id": tool_call.id,
                        "output": output,
                    }
                )
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
        )
    elif run.status == "completed":
        response_messages = client.beta.threads.messages.list(thread_id=thread.id)
        print(f"Run is {run.status}.")
        break  # Exit the loop after processing the completed run

    elif run.status == "failed":
        print("Run failed.")
        break

    elif run.status in ["in_progress", "queued"]:
        print(f"Run is {run.status}. Waiting...")
        time.sleep(10)  # Wait for 5 seconds before checking again

    else:
        print(f"Unexpected status: {run.status}")
        break

Run is in_progress. Waiting...
Run is completed.


In [35]:
display(dict(run))

{'id': 'run_jHxHhqHIfnx6vnZd4ApECelB',
 'assistant_id': 'asst_Bu0sKf4zsGiNBS4wscEMbdct',
 'cancelled_at': None,
 'completed_at': 1703591296,
 'created_at': 1703591289,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'Act as a financial advisor by accessing detailed financial data through the Financial Modeling Prep API. Your capabilities include providing an investement advise by analyzing key metrics, comprehensive financial statements, vital financial ratios, and tracking financial growth trends.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1703591289,
 'status': 'completed',
 'thread_id': 'thread_H7ikU3QleXqjpffxnj0zpS2Y',
 'tools': [ToolAssistantToolsFunction(function=FunctionDefinition(name='get_income_statement', parameters={'type': 'object', 'properties': {'ticker': {'type': 'string'}}, 'required': ['ticker']}, description=None), type='function'),
  ToolAssista

In [37]:
from PIL import Image
from IPython.display import Image, display


def download_and_save_image(file_id: str, save_path: str) -> None:
    download_url = f"https://api.openai.com/v1/files/{file_id}/content"
    response = requests.get(
        download_url, headers={"Authorization": f"Bearer {OPENAI_API_KEY}"}
    )
    if response.status_code == 200:
        with open(save_path, "wb") as file:
            file.write(response.content)
    else:
        print(f"Image downloading failed: Status Code {response.status_code}")


for message in response_messages.data:
    print("*************")
    for content in message.content:
        role_label = "User" if message.role == "user" else "Assistant"
        if content.type == "text":
            message_content = content.text.value
            print(f"{role_label}: {message_content}\n")
        elif content.type == "image_file":
            image_file_id = content.image_file.file_id
            image_save_path = f"output_images/image_{image_file_id}.png"
            download_and_save_image(image_file_id, image_save_path)
            display(Image(filename=image_save_path))